In [1]:
import numpy as np
import pandas as pd

import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer

import os
import re

In [2]:
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/denisshpilka/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/denisshpilka/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [ ]:
train_path = os.path.join('data', 'train.csv')
target_columns = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']

df = pd.read_csv(train_path)
df

In [ ]:
stemmer = SnowballStemmer(language='english')
stopwords = stopwords.words('english')

def preprocess_text(text):
    text_tokenized = word_tokenize(text)
    
    stopwords = stopwords.words('english')
    text_stp = [word for word in text_tokenized if (word.lower() not in stopwords) and word.isalpha()]
    
    stemmer = SnowballStemmer(language='english')
    processed_text = [stemmer.stem(word) for word in text_stp]

    return processed_text